In [1]:
import sys, os, json, torch, numpy as np
import importlib
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use("Agg")  # headless mode

# --- Setup ---
sys.path.insert(0, "/work/gbadarac/MonoJet_NPLM/MonoJet_NPLM_analysis/Train_Ensembles/Train_Models")
import utils_flows
importlib.reload(utils_flows)
from utils_flows import make_flow, make_flow_zuko, plot_individual_marginals

# === Trial name and paths ===
trial_name = "N_100000_dim_2_seeds_60_4_16_128_15"

base_dir = "/work/gbadarac/MonoJet_NPLM/MonoJet_NPLM_analysis/Train_Ensembles/Train_Models/Normalizing_Flows/nflows/EstimationNFnflows_outputs/2_dim/2d_bimodal_gaussian_heavy_tail"
trial_dir = os.path.join(base_dir, trial_name)
f_i_file = os.path.join(trial_dir, "f_i.pth")

out_dir = os.path.join(trial_dir, "marginals_80_bins")
os.makedirs(out_dir, exist_ok=True)

# === Load architecture config ===
with open(os.path.join(trial_dir, "architecture_config.json")) as f:
    config = json.load(f)

backend = config.get("backend", "nflows")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# === Load reference data ===
data_path = "/work/gbadarac/MonoJet_NPLM/MonoJet_NPLM_analysis/Train_Ensembles/Generate_Data/saved_generated_target_data/2_dim/100k_2d_gaussian_heavy_tail_target_set.npy"
x_data = torch.from_numpy(np.load(data_path)).float().to(device)

# === Reconstruct f_i models ===
# === Choose which models to plot ===
model_indices = range(10)  # all models
# === Load only selected models ===
f_i_models = []
for i in model_indices:
    model_path = os.path.join(trial_dir, f"model_{i:03d}", "model.pth")
    state_dict = torch.load(model_path, map_location=device)

    flow = (
        make_flow_zuko(
            num_layers=config["num_layers"],
            hidden_features=config["hidden_features"],
            num_bins=config["num_bins"],
            num_blocks=config["num_blocks"],
            num_features=2,
            num_context=0,
            bayesian=config.get("bayesian", False)
        )
        if backend == "zuko"
        else make_flow(
            num_layers=config["num_layers"],
            hidden_features=config["hidden_features"],
            num_bins=config["num_bins"],
            num_blocks=config["num_blocks"],
            num_features=config["num_features"]
        )
    ).to(device)

    flow.load_state_dict(state_dict)
    flow.eval()
    f_i_models.append(flow)

feature_names = ["Feature 1", "Feature 2"]
plot_individual_marginals(
    f_i_models,
    x_data,
    feature_names,
    out_dir,
    num_samples=20000,
    use_zuko=(backend == "zuko")
)